# Evaluate the performance of a trained model

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
from torch.utils.data import DataLoader
from torch.nn import DataParallel

from vos.models.STM import STM
from vos.algo.stm_train import STMAlgo
from vos.datasets.DAVIS import DAVIS_2017_TrainVal
from vos.utils.helpers import stack_images, stack_masks

In [ ]:
davisroot = "/p300/videoObjSeg_dataset/DAVIS-2017-trainval-480p"
videod = DAVIS_2017_TrainVal(davisroot, mode= "val")
dataloader = DataLoader(videod)

In [ ]:
statedict_root = "/p300/VideoObjSeg_data/local/video_segmentation/20200311/affine0.0-10.0-0.05-0.0/img_res-384,384/NNSTM/big_objects-True3/b_size-1/run_0"
state_dict = torch.load(os.path.join(statedict_root, "params.pkl"))
model_state_dict = state_dict['model_state_dict']

In [ ]:
model = DataParallel(STM())
model.load_state_dict(model_state_dict)
model.eval()

In [ ]:
algo = STMAlgo(
    eval_step_kwargs= dict(Mem_every= 5),
)
algo.initialize(model)

## Start evaluating and calculating the mean loss and performance

In [ ]:
loss, IoU = 0.,0.
for i, data in enumerate(dataloader):
    print(f"Evaluating at itereation: {i}, loss: {loss/(i+1)}, IoU: {IoU/(i+1)}", end= "\r")
    eval_info, extra_info = algo.eval(i, data)
    loss += eval_info["loss"]
    IoU += eval_info["IoU"]